In [11]:
# How TF - IDF works
import re 
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer


texts = ['hello how are you',
        'how are you doing',
         'thank you i am doing fine',
         'hope you are doing well'
        ]

vectorizer = TfidfVectorizer()
response = vectorizer.fit_transform(texts)
terms = vectorizer.get_feature_names()

tfidf_sklearn = pd.DataFrame(response.toarray(), columns=terms)
chosen_words = list(tfidf_sklearn.columns)
tfidf_sklearn

,am,are,doing,fine,hello,hope,how,thank,well,you
0,0.000000,0.420753,0.000000,0.000000,0.659191,0.000000,0.519714,0.000000,0.000000,0.343993
1,0.000000,0.488291,0.488291,0.000000,0.000000,0.000000,0.603137,0.000000,0.000000,0.399210
2,0.521305,0.000000,0.332742,0.521305,0.000000,0.000000,0.000000,0.521305,0.000000,0.272039
3,0.000000,0.363277,0.363277,0.000000,0.000000,0.569144,0.000000,0.000000,0.569144,0.297003


# How did we get the above calculation
# Lets take row 3 for example - How did we get the value of 0.538029 for "hope" ?

In [ ]:
# Term Frequency (word, doc) -> (Number of times doing appears in document 3)/ (Total Number of words in document3)
# Inverse Document Frequency (word) -> log((Number of documents)+1/ (Number of documents containing the word)+1)+1

In [ ]:
# Term Frequency
tf = 1/4

In [ ]:
# IDF
idf = np.log((4+1)/(1+1))+1

In [ ]:
# Product of TF-IDF
product = tf * idf
print(product)

In [ ]:
# create dictionary
def calculate_tf(word, document):
    count = 0
    text_split = document.split(' ')
    for each_word in text_split:
        if word == each_word:
            count+=1
    return count/len(text_split)

def calculate_idf(word):
    number_of_occurences_of_word_in_all_documents = 0
    for text in texts:
        text_split = text.split(' ')
        if word in text_split:
            number_of_occurences_of_word_in_all_documents+=1
    return np.log((len(texts)+1)/(number_of_occurences_of_word_in_all_documents+1))+1


#chosen_words = ['doing', 'fine', 'hello', 'hope', 'thank']
dfs = []
for index, text in enumerate(texts):
    temp_dicts = []
    for word in chosen_words:
        temp_dict = {}
        temp_dict['word'] = word     
        temp_dict['tf'] = calculate_tf(word, text)          
        temp_dicts.append(temp_dict)
    temp_df = pd.DataFrame(temp_dicts).T
    temp_df.columns = chosen_words    
    dfs.append(temp_df[1:])
tfidf_calc = pd.concat(dfs)
tfidf_calc.reset_index(drop = True, inplace = True)
tfidf_calc

In [ ]:
# calculate IDF matrix
import pandas as pd
cv = CountVectorizer()
word_count_vector = cv.fit_transform(texts)


tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(word_count_vector)
idf = pd.DataFrame({'feature_name':cv.get_feature_names(), 'idf_weights':tfidf_transformer.idf_}).T[1:]
idf.columns = chosen_words
idf

In [ ]:
# Multiple TF-IDF

sum_of_squares = []
for index, row in tfidf_calc.iterrows():
    sum_square = 0
    for column in tfidf_calc.columns:
        row[column] = row[column] * idf[column][0]
        sum_square+=np.square(row[column])
    sum_of_squares.append(np.sqrt(sum_square))


tfidf_calc['sum_of_squares'] = sum_of_squares
tfidf_calc  

In [ ]:
word_columns = list(tfidf_calc.columns)
word_columns.remove('sum_of_squares')
for index, row in tfidf_calc.iterrows():
    for column in word_columns:
        row[column] = row[column]/row['sum_of_squares']
        tfidf_calc.iloc[index] = row
tfidf_calc

In [ ]:
tfidf_sklearn

# Now we can see the values matches with what we get from SKLEARN